<a href="https://colab.research.google.com/github/YianKim/2022_PythonStudy_face_emotion_recognition/blob/main/uncertainty_%EA%B8%B0%EB%B0%98_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글 드라이브


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 라이브러리 불러오기


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import PIL
import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import LSTM
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import *
from keras.models import load_model

from tqdm import tqdm

## 데이터 불러오기


In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(data, labels), (test_data, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [4]:
train_data = data[range(10000)].reshape([10000,28,28,1])
train_labels = labels[range(10000)].reshape([10000,1])

In [5]:
unlab_data = data[range(10000,60000)].reshape([50000,28,28,1])
unlab_labels = labels[range(10000,60000)].reshape([50000,1])

In [6]:
test_data = test_data.reshape([10000,28,28,1])
test_labels = test_labels.reshape([10000,1])

In [7]:
train_labels2 = []
unlab_labels2 = []
test_labels2 = []

In [8]:
for i in range(10000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[train_labels[i][0]] = 1
  train_labels2.append(white)

for i in range(50000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[unlab_labels[i][0]] = 1
  unlab_labels2.append(white)

for i in range(10000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[test_labels[i][0]] = 1
  test_labels2.append(white)

In [9]:
train_labels = np.array(train_labels2)
unlab_labels = np.array(unlab_labels2)
test_labels = np.array(test_labels2)

## CNN

In [10]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [11]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
110/110 [==============================] - 15s 18ms/step - loss: 3.2566 - accuracy: 0.5900 - val_loss: 0.8345 - val_accuracy: 0.7040 - lr: 0.0010
Epoch 2/2000
110/110 [==============================] - 2s 14ms/step - loss: 0.6978 - accuracy: 0.7519 - val_loss: 0.6845 - val_accuracy: 0.7440 - lr: 0.0010
Epoch 3/2000
110/110 [==============================] - 1s 13ms/step - loss: 0.5848 - accuracy: 0.7866 - val_loss: 0.6348 - val_accuracy: 0.7847 - lr: 0.0010
Epoch 4/2000
110/110 [==============================] - 2s 14ms/step - loss: 0.5179 - accuracy: 0.8086 - val_loss: 0.6325 - val_accuracy: 0.7720 - lr: 0.0010
Epoch 5/2000
110/110 [==============================] - 2s 14ms/step - loss: 0.4800 - accuracy: 0.8259 - val_loss: 0.6128 - val_accuracy: 0.7900 - lr: 0.0010
Epoch 6/2000
110/110 [==============================] - 1s 13ms/step - loss: 0.4445 - accuracy: 0.8356 - val_loss: 0.5553 - val_accuracy: 0.8027 - lr: 0.0010
Epoch 7/2000
110/110 [=============================

In [12]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [13]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

## 모델 결과

In [14]:
def model_eval():
  pred_mu = model.predict(test_data)
  for i in range(1, 30):
    pred_mu += model.predict(test_data)
  pred_mu = pred_mu/30
  predicted_test_labels = np.argmax(pred_mu, axis=1)
  return(accuracy_score(np.argmax(test_labels, axis=1), predicted_test_labels))

In [15]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')
model_eval()

0.8871

# 라벨링

## 라벨링; 1

In [16]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [17]:
# Vars : 분산들
# Outs : 결과 값들

Vars = []
Outs = []

temp1 = []

for i in tqdm(range(30)):
  temp1.append(model.predict(unlab_data))

for j in tqdm(range(unlab_data.shape[0])):

  temp2 = np.array([0,0,0,0,0,0,0,0,0,0], 'float32')
  
  for i in range(30):
    temp2 += temp1[i][j]
  Outs.append(temp2/30)
  
  outputs=[]

  for i in range(30):
    outputs.append(temp1[i][j][np.argmax(temp2)])
  Vars.append(np.var(outputs))

100%|██████████| 50000/50000 [00:10<00:00, 4597.56it/s]


In [18]:
Outs = np.array(Outs)

class마다 균등하게 선택

In [19]:
Counter(np.argmax(unlab_labels, axis=1))

Counter({0: 5058,
         1: 4973,
         2: 4984,
         3: 4981,
         4: 5026,
         5: 5011,
         6: 4979,
         7: 4978,
         8: 5010,
         9: 5000})

In [20]:
# upper bound of variance?

UB = []

for h in range(10):
  classvars = []
  for i in tqdm(range(50000)):
    if np.argmax(unlab_labels, axis=1).tolist()[i]==h:
      classvars.append(Vars[i])
  UB.append(np.percentile(classvars, 50))

100%|██████████| 50000/50000 [01:31<00:00, 544.48it/s]


In [21]:
lowvars = []
ind = 0 

for i in Vars:
  if i <= UB[np.argmax(unlab_labels, axis=1)[ind]]:
    lowvars.append(ind)
  ind += 1

In [22]:
highvars = []
for i in range(unlab_data.shape[0]):
  if i not in lowvars:
    highvars.append(i)

In [23]:
# 같은 길이의 랜덤하게 고른 data들에 모델로 label 부여 후 정확도 측정
randomindx = random.sample(range(50000), len(lowvars))
accuracy_score(np.argmax(np.array(Outs)[randomindx], axis=1), np.argmax(unlab_labels[randomindx], axis=1))

0.8926657602175478

In [24]:
# 저분산의 data들만 모아서 모델로 label 부여 후 정확도 측정 : 95% 이상
accuracy_score(np.argmax(np.array(Outs)[lowvars], axis=1), np.argmax(unlab_labels[lowvars], axis=1))

0.972726545629049

In [25]:
train_data_1 = np.concatenate([train_data, unlab_data[lowvars]], axis=0)
train_labels_1 = np.concatenate([train_labels, Outs[lowvars]], axis=0)

In [26]:
shufindx = random.sample(range(train_data_1.shape[0]),train_data_1.shape[0])
train_data_1 = train_data_1[shufindx]
train_labels_1 = train_labels_1[shufindx]

In [27]:
train_data_2 = np.concatenate([train_data, unlab_data[randomindx]], axis=0)
train_labels_2 = np.concatenate([train_labels, Outs[randomindx]], axis=0)

In [28]:
shufindx = random.sample(range(train_data_2.shape[0]),train_data_2.shape[0])
train_data_2 = train_data_2[shufindx]
train_labels_2 = train_labels_2[shufindx]

In [29]:
# unlab_data = unlab_data[highvars]
# unlab_labels = unlab_labels[highvars]

In [30]:
Counter(np.argmax(train_labels_1, axis=1))

Counter({0: 3629,
         1: 3523,
         2: 3613,
         3: 3633,
         4: 3466,
         5: 3491,
         6: 3130,
         7: 3506,
         8: 3506,
         9: 3509})

In [31]:
Counter(np.argmax(train_labels_2, axis=1))

Counter({0: 3447,
         1: 3439,
         2: 3556,
         3: 3547,
         4: 3489,
         5: 3416,
         6: 3457,
         7: 3600,
         8: 3530,
         9: 3525})

## 모델링

간단한 모델에서 받은 지식를 복잡한 모델에서 학습 

uncertainty aware data vs random sampling data

In [32]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [33]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)


model.fit(
          train_data_1,
          train_labels_1,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
383/383 [==============================] - 10s 24ms/step - loss: 0.9095 - accuracy: 0.7146 - val_loss: 0.4576 - val_accuracy: 0.8456 - lr: 0.0010
Epoch 2/2000
383/383 [==============================] - 9s 23ms/step - loss: 0.4123 - accuracy: 0.8658 - val_loss: 0.3764 - val_accuracy: 0.8777 - lr: 0.0010
Epoch 3/2000
383/383 [==============================] - 9s 23ms/step - loss: 0.3455 - accuracy: 0.8906 - val_loss: 0.3149 - val_accuracy: 0.9012 - lr: 0.0010
Epoch 4/2000
383/383 [==============================] - 9s 23ms/step - loss: 0.2991 - accuracy: 0.9072 - val_loss: 0.2862 - val_accuracy: 0.9136 - lr: 0.0010
Epoch 5/2000
383/383 [==============================] - 9s 22ms/step - loss: 0.2782 - accuracy: 0.9141 - val_loss: 0.3001 - val_accuracy: 0.9114 - lr: 0.0010
Epoch 6/2000
383/383 [==============================] - 9s 23ms/step - loss: 0.2581 - accuracy: 0.9211 - val_loss: 0.2684 - val_accuracy: 0.9206 - lr: 0.0010
Epoch 7/2000
383/383 [=============================

In [34]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')

In [35]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')
model_eval()

0.8946

In [36]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################

model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data_2,
          train_labels_2,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
383/383 [==============================] - 10s 23ms/step - loss: 1.2357 - accuracy: 0.5925 - val_loss: 0.7226 - val_accuracy: 0.7649 - lr: 0.0010
Epoch 2/2000
383/383 [==============================] - 9s 22ms/step - loss: 0.6191 - accuracy: 0.7907 - val_loss: 0.5807 - val_accuracy: 0.8062 - lr: 0.0010
Epoch 3/2000
383/383 [==============================] - 9s 23ms/step - loss: 0.5445 - accuracy: 0.8221 - val_loss: 0.5088 - val_accuracy: 0.8370 - lr: 0.0010
Epoch 4/2000
383/383 [==============================] - 9s 23ms/step - loss: 0.4895 - accuracy: 0.8467 - val_loss: 0.4802 - val_accuracy: 0.8510 - lr: 0.0010
Epoch 5/2000
383/383 [==============================] - 9s 22ms/step - loss: 0.4571 - accuracy: 0.8608 - val_loss: 0.4704 - val_accuracy: 0.8522 - lr: 0.0010
Epoch 6/2000
383/383 [==============================] - 9s 22ms/step - loss: 0.4338 - accuracy: 0.8697 - val_loss: 0.4286 - val_accuracy: 0.8721 - lr: 0.0010
Epoch 7/2000
383/383 [=============================

In [37]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')

In [38]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')
model_eval()

0.8928